In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import json
from pymongo import MongoClient
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
consumer_key = "xxxxxx"
consumer_secret = "xxxxxx"
acess_token = "xxxxxx"
acess_token_secret = "xxxxxx"

In [ ]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_acess_token(acess_token, acess_token_secret)


In [ ]:
class MyListener(StreamListener):
    def on_data(self,data):
        tweet = json.loads(data)
        created_at = tweet["created_at"]
        id_str = tweet[ "id_str"]
        text = tweet[ "text"]
        obj = {"created_at":created_at, "id_str":id_str, "text":text}
        tweetind = col.insert_one(obj).inserted_id
        print(obj)
        return True

In [ ]:
mylistener = MyListener()

In [ ]:
mystream = Stream(auth, listener = mylistener)

In [ ]:
client = MongoClient('localhost', 27017)
db = client.twitterdb
col = db.tweets
keywords = ['bieber']

In [ ]:
mystream.filter(track=keywords)

In [ ]:
col.find_one()

In [ ]:
dataset = [{"created_at":item["created_at"],
           "text":item["text"],
           } for item in col.find()]

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
df.head()

In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [ ]:
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:50]